- summary 
- web : server-client : url 
- requests : 동적페이지 : URL 변화 없이 페이지의 데이터 수정 : json(str) > response.json() > DataFrame 
- requests : 정적페이지 : URL 변화 있이 페이지의 데이터 수정 : html(str) > BeautifulSoup > css-selector > DataFrame 
- selenium : 웹브라우져를 python 코드로 컨트롤해서 데이터 수집 
- requests(동적페이지,API) > requests(정적페이지) > selenium 
- 웹크롤링 절차 
- 1. 웹서비스분석(개발자도구), API문서 : URL 
- 2. request(URL) > response(data) : data(json(str), html(str)) 
- 3. data(json(str), html(str)) > response.json(), BeautifulSoup(css-selector) > DataFrame 
- request 할때 401,403,500 등등의 에러가 발생하는 경우 > headers 수정해서 데이터 요청(user-agent, refere) 
- API 이용 : request token 수집후 크롤링

### Zigbang 원룸 매물 데이터 수집

##### 동 이름 입력하면 > 해당 동의 매물 데이터가 나오겠금

In [1]:
import pandas as pd
import requests

#### Process
    - 동이름으로 위도 경도 구하기(직빵엔 매물 데이터가 없음...)
    - 위도 경도로 geohash 알아내기
    - geohash로 매물 아이디 가져오기
    - 매물 아이디로 매물 정보 가져오기

#### 1. 동이름으로 위도 경도 구하기

- address url은 https://meyerweb.com/eric/tools/dencoder/ 에서 decode 해가지고 가져왔음

In [7]:
# 1. URL
address = '상인'
url = f'https://apis.zigbang.com/v2/search?leaseYn=N&q={address}&serviceType=원룸'
# 2. request > response : json(str)
response = requests.get(url)
# 3. json(str) > lat, lng(위도값, 경도값)

In [8]:
response.text

'{"success":true,"code":"200","items":[{"id":2865,"type":"address","name":"상인동","hint":"","description":"대구시 달서구 상인동","lat":35.81401062011719,"lng":128.5452880859375,"zoom":5,"polygon":[],"_score":null,"_source":{"name_length":3,"local1":"대구시","local2":"달서구","local3":"상인동","web_level":15,"web_lat":35.81401062011719,"web_lng":128.5452880859375,"app_level":15,"app_lat":35.81401062011719,"app_lng":128.5452880859375,"법정동코드":"2729011500"},"zoom_level":{"google":15,"daum":4},"zoom_level_v2":{"app":5,"web":4}},{"id":2862,"type":"address","name":"상인1동","hint":"","description":"대구시 달서구 상인1동","lat":35.81401062011719,"lng":128.5452880859375,"zoom":5,"polygon":[],"_score":null,"_source":{"name_length":4,"local1":"대구시","local2":"달서구","local3":"상인1동","web_level":15,"web_lat":35.81401062011719,"web_lng":128.5452880859375,"app_level":15,"app_lat":35.81401062011719,"app_lng":128.5452880859375,"법정동코드":null},"zoom_level":{"google":15,"daum":4},"zoom_level_v2":{"app":5,"web":4}},{"id":2863,"type":"address

In [10]:
# list > dict > items만 가져오기
data = response.json()['items'][0]
lat, lng = data['lat'], data['lng']
lat, lng

(35.81401062011719, 128.5452880859375)

#### 2. 위도 경도로 geohash 알아내기

#### 3. geohash로 매물 아이디 가져오기

#### 4. 매물 아이디로 매물 정보 가져오기